# 基于类别的情感分析系统搭建
> 从0开始搭建情感分析系统

- toc: true 
- badges: true
- comments: true
- author: Bujie Xu
- categories: [NLP]

## 1. 项目背景和问题描述

维基百科的定义里说，文本情感分析是指用自然语言处理、文本挖掘以及计算机语言学等方法来识别和提取原素材中的主观信息。情感分析的目的是找出作者对某个事物的观点。很多人在网上购物或者在外就餐之后，都会通过发表评论来描述自己的体验。对于商家来说，分析这些文本就可以较好的看清楚用户的反馈，从而提升服务质量。另外，在投资领域，情感分析也有所应用。通过分析twitter上对某家公司的整体情感，可以给出大众对这家公司的一个整体看法，从而对投资起到一个指导作用。

本文将介绍怎么样从0开始搭建一个简单的情感分析系统。所用的到的数据来自[Yelp Dataset](https://www.yelp.com/dataset/documentation/main), Yelp对标国内的大众点评，这个数据集里收集了很多的用户评论信息。下图是来自yelp网站的一个例子。

![yelp](images/sentiment1.PNG)

在Review Highlights里，我们可以看到一些用户评价，而且Yelp还对评价进行了归类，比如第一类是有对souvenirs的评价，第二类是对electronics的评价。这里的souvenirs和electronics我们可以看作是商品或者是服务的一个维度(Aspect)。我们要搭建的情感分析系统要在传统的情感分析系统上做一点提高。

我们不断要判断用户评价是否是积极的，而且我们要从用户评价中获取用户是对哪个商品维度所进行的描述，并判断用户对每一个商品维度的情感。
对于每一条评价，我们想通过系统获取如下结果。

`
{
    Business Name: XXXXX, 
    Overall Rating: X,
    aspect1: { rating: XXX, pos: [XXX], neg: [XXX]}
    aspect2: {rating: XXX, pos: [XXX], neg: [XXX]}
    aspect3: {rating: XXX, pos: [XXX], neg: [XXX]} ……
    aspect5: {rating: XXX, pos:[xxx], neg:[xxxx]}
}
`


## 2. baseline搭建

在这里，我们用到yelp数据集里的“yelp_academic_dataset_business”和“yelp_academic_dataset_review”。可以通过这个链接查看[样本数据](https://www.yelp.com/dataset/documentation/main)。

我们的baseline包含以下几个部分

1. 获取每个business的overall Rating
2. 获取每一个business的aspect
3. 获取对于aspect的评价
    1. 对于每个aspect的综合评价
    2. 从用户评论中获得对于某个aspect的相关内容，并进行分类（判断是好评还是差评）

#### 获取Overall Rating

我们只需要对每个business的所有评论的结果求平均即可得到Overall Rating
```python
for Id in business_id:
    star = []
    for review in reviews[Id]:
        star.append(review['stars'])
    result[Id] = star.mean()
        
```

#### 获取每一个business的aspect

提取aspect，我们采用最简单的方法，对于每一个business，循环所有评论，统计评论中的名字词频，在去除stopwords之后，找到排名前5的词语，即作为该business的top 5 aspects。

```python
# 从每个评论中提取名词
def extract_and_add_aspects(aspects_dict, review):
    business_id = review['business_id']
    text = review['text'].lower()
    text = word_tokenize(text)
    aspects = [tag[0] for tag in nltk.pos_tag(text,tagset='universal') if tag[1] == 'NOUN']
    aspects_dict[business_id].append(aspects)

# 对于每一个business，提取在评论中词频最高的5个名词（去除stopword之后）
def get_top_5_aspects(aspects_dict, business_id):
    l = list(itertools.chain.from_iterable(aspects_dict[business_id]))
    sorted_l = Counter(l).most_common()
    top_5_aspects = []
    for s in sorted_l:
        if len(top_5_aspects) == 5:
            break
        elif s[0] in stopwords.words('english'):
            continue
        else:
            top_5_aspects.append(s[0])
    return top_5_aspects
```

#### 获取对于aspect的评价

为了获取对每一个aspect的评价，我们首先需要训练一个情感分析的模型。由于是baseline，我们首先尝试一个简单的模型来建立这个情感分析模型。本文采用tfidf vectorizer+random forest来建模。详细步骤分为以下几步
1. 对所有的review中的文本（text）进行tfidf变换
2. 将tfidf生成的矩阵作为输入，review的stars作为输出进行模型的训练。
3. 用random forest来模型进行训练。因为这里的stars的取值为[0,1,2,3,4,5],　是一个有序的分类问题，我们也可以把该问题看成是一个回归问题，用regressor的效果会更好

```python
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
tidif_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
review_tidif_vector = tidif_vectorizer.fit_transform(df_review['text'])
clf.fit(review_tidif_vector, df_review['stars'])
```

训练完模型之后，我们还需要做关键的另一步，就是从用户评论中获取跟对应aspect相关的部分。这里，我们采取最简单的方法，首先按标点符号(这里我们只用了',.!')将用户评价进行切分。然后对于每个aspect，如果分句中出现了他，则判断该分局与这个aspect有关。

```python
def get_segment(aspect, text):
    '''
    Input an aspect and a text, the function will return the setences contain the aspect,
    if there are no such aspec the segment could be empty list.
    '''
    setences = re.split('[,.!]', text)
    segment = []
    for s in setences:
        if aspect in s:
            segment.append(s.strip())
    return segment
```

完成了上两步后，我们既可以对每一个business的每一个review进行分析，求得最终结果。

```python
for review in business_id.get_reviews():
    if review.contains(aspect1):
    review_segment = get_segment(review, aspect)
    score = sentiment_model.predict_prob(review_segment)
    if score > threshold:
        pos_sent[review] = score
    else:
        neg_sent[review] = score
```

在文本数据的预处理方面，常用的还有count vectorizer和word embedding， count vectorizer的效果比起tfidf略微差了一点，word embedding对结果会有所提高。另外，在情感分析模型的构建中，还可以有很多其他的方法，例如目前最流行的Bert。或者也可以利用LSTM来构建模型。在这方面，如果有兴趣可以参考https://www.kaggle.com/poonaml/bidirectional-lstm-spacy-on-yelp-reviews。 结果会比baseline有很大的提升。不过如果想要尝试这类深度学习的方法，必须先准备好GPU。

对于Aspect的抽取，Segment的抽取。本文的方法比较基础，这是考虑了最基础的情况。这些方面的抽取也可以考虑通过训练模型来实现。

## 4. 结果

以下给出一个对于某个business的结果精选。

========= sample result for business_id jRfQX8enRhWHf7V5zP5U8g =========

Business Name:  The Gelato Spot

Overall Rating:  4.058315334773218

Aspects1 :gelato, rating:3.7201255688450567, 

pos_sample: ['This review is only with respect to their gelato', "We haven't yet ventured beyond gelato but will make it our next stop when we're in the mood for pizza", 'The gelato LOOKS delicious from the get-go', 'The guy behind the gelato counter was very helpful and friendly', 'I saw a decent amount of customer flow and the employees were constantly making gelato in the back'], 

neg_sample: ["Worst gelato I've had", 'The girls serving gelato had horrible attitudes and kept pushing me off to each other (they openly said they did not want to wait on me and told each other to just have 
me wait and to help someone else)', 'their gelato was nothing special']

Aspects2 :pizza, rating:3.6547648329047857, 

pos_sample: ["We haven't yet ventured beyond gelato but will make it our next stop when we're in the mood for pizza", "one of these days I'll try the pizza place as well and I'm sure it won't disappoint", 'Vegetarian Friendly Entree: Honey Basil Pizza\nDelicious pizza with the sweet and savory combination', 'One pizza is a perfect serving for one person', "I'm the type of 
person that can never choose the type of pizza I want"], 

neg_sample: ['Showed up Saturday night to order a pizza and was told I could order but had to take it to go since they were closing in 30 minutes', 'Was told it took 15 minutes for the pizza to be ready', 'Worst pizza I have ever had', 'Lean cuisine pizzas have more flavor', "the margherite pizza was the worst I've had"]

Aspects3 :place, rating:3.671313509696845, 

pos_sample: ['The place is very clean and nicely decorated', 'This place is great', 'The place is conveniently located in a corner location within a pizzeria', 'The location is clean and the place is quaint and comfortable', 'I frequent this place more often than I care to admit but with free samples and a great variety of different gelato flavors how could I not? Some common reasons why you might find me in Gelato Spot:\n\n- Holiday\n- Bad Day\n- Good Day\n- Normal Day\n- Sunday'], neg_sample: ['I decided to order pick up using the Eat2 services for lunch since my work place is walking distance from the restaurant', 'After helping them I told her I was there to pick up an order i placed using the Yelp app', 'I showed her the charge from my bank account and she placed the order to the cooks', 'I had to rush back to my work place since I only have a 30min lunch']

Aspects4 :flavors, rating:3.6733356958870567, 

pos_sample: ['Great flavors and good quality', 'Their flavors are all delicious', 'and they are great at suggesting new flavors', 'and they have so many flavors to choose from', "We haven't been disappointed in any flavors that we've tried:  tiramisu"], 

neg_sample: []

Aspects5 :spot, rating:3.6346406123098576, 

pos_sample: ['I was pleasantly surprised with this spot', 'It hit the spot', 'I am so happy that we found this spot and if I ever come back', 'This review is for Wood Fired pizza which is connected to the gelato spot', 'The G spot has it all'], 

neg_sample: ['My daughter read about their cakes on their website so when I asked her where I should get her birthday cake she told me that Gelato spot had cakes with buttercream frosting(her favorite)']


## 4. 其他尝试

搭建一个情感分析系统需要牵扯到很多方面，任何一个环节都需要注意。如果有一个环节出错，都会影响到整个系统的表现。例如Aspect的抽取，Segment的抽取，或者是模型的构建。

{{ '[文本情感分析](https://zh.wikipedia.org/wiki/%E6%96%87%E6%9C%AC%E6%83%85%E6%84%9F%E5%88%86%E6%9E%90)' | fndetail: 1 }}
{{ '[Bidirectional LSTM, SpaCy on Yelp Reviews](https://www.kaggle.com/poonaml/bidirectional-lstm-spacy-on-yelp-reviews)' | fndetail: 2 }}
{{ '[Yelp Dataset](https://www.yelp.com/dataset/documentation/main)' | fndetail: 3 }}